In [ ]:
import umap

import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI

from scipy import sparse
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
rcParams['figure.figsize'] = (12,12)
import seaborn as sns
from scipy import sparse
# 
sc.settings.verbosity =0

import matplotlib.pyplot as plt
import pandas as pd
import os
compute = True

import numpy as np
import pandas as pd
from scipy.sparse.csgraph import connected_components

    
def binary_LISI(neigh, metadata):
    """
    Compute a binary version of the Local Inverse Simpson Index (LISI).

    :param neigh: A pynndescent object with the 'indices' attribute having shape (n_cells, n_neighbors)
    :param metadata: An array of shape (n_cells,) containing metadata labels for each cell
    :return: A numpy array of shape (n_cells,) with the binary LISI scores
    """
    n_cells = neigh.indices.shape[0]
    binary_lisi_scores = np.zeros(n_cells)

    for i in range(n_cells):
        center_label = metadata[i]
        neighbor_indices = neigh.indices[i]
        neighbor_labels = metadata[neighbor_indices]

        # Count how many neighbors have the same label as the center cell
        same_label_count = np.sum(neighbor_labels == center_label)
        
        # Compute the proportion of same label in the neighborhood
        binary_lisi_scores[i] = same_label_count / len(neighbor_indices)

    return binary_lisi_scores

def result_exists(mod, i, ann, n_size, ct):
    ct_sanitized = ct.replace('/', '_')  # Sanitize cell type name for use in filename
    filename = f'CLISI_csv_PARTIAL/scIB_{mod}_{i}_{ann}_BINARY_{n_size}_{ct_sanitized}.csv'
    return os.path.exists(filename), filename

def save_binary_lisi_scores(binary_lisi_scores, filename, valid_indices):
    temp_df = pd.DataFrame(binary_lisi_scores, index=valid_indices, columns=['binary_LISI'])
    print(temp_df.shape)

    temp_df.to_csv(filename)



In [2]:
l1_colors = {
'ATL': '#1f77b4',
  'PT_VCAM1': '#c5b0d5',
 'CNT': '#ff7f0e',
 'DCT': '#279e68',
  'DCT1': '#279e68',
 'DCT2': '#ffb500',
 'DTL': '#d62728',
 'EC': '#aa40fc',
 'ENDO': '#aa40fc',
 'FIB': '#8c564b',
 'IC': '#e377c2',
 'ICA': '#e377c2',
 'ICB': '#7b4f4b',
 'IMM': '#b5bd61',
 'LEUK': '#b5bd61',
 'NEU': '#17becf',
 'PC': '#aec7e8',
 'PEC': '#17becf',
 'POD': '#98df8a',
 'PODO': '#98df8a',
'PT': '#ff9896',
 'PapE': '#c5b0d5',
 'TAL': '#c49c94',
'MES': '#f7b6d2',
 'VSM/P': '#f7b6d2','Unclassified':'#d3d3d3'}

In [3]:
l3_colors = {
'ATL': '#ffff00',
 'B': '#1ce6ff',
 'C-TAL': '#ff34ff',
 'PC': '#aec7e8',
 'CCD-IC-A': '#ff4a46',
 'CCD-PC': '#008941',
 'CNT': '#006fa6',
 'CNT-IC-A': '#a30059',
 'CNT-PC': '#ffdbe5',
 'DCT1': '#7a4900',
 'DCT2': '#0000a6',
 'DTL': '#63ffac',
 'EC-AEA': '#b79762',
 'EC-AVR': '#004d43',
 'EC-DVR': '#8fb0ff',
 'EC-GC': '#997d87',
 'EC-LYM': '#5a0007',
 'EC-PTC': '#809693',
 'FIB': '#6a3a4c',
 'IC-B': '#1b4400',
 'IMCD': '#4fc601',
 'M-FIB': '#3b5dff',
 'M-TAL': '#4a3b53',
 'MAC-M2': '#ff2f80',
 'MAST': '#61615a',
 'MC': '#ba0900',
 'MD': '#6b7900',
 'MDC': '#00c2a0',
 'MYOF': '#ffaa92',
 'N': '#ff90c9',
 'NEU': '#b903aa',
 'NKC/T': '#d16100',
 'OMCD-IC-A': '#ddefff',
 'OMCD-PC': '#000035',
 'PEC': '#7b4f4b',
 'PL': '#a1c299',
 'POD': '#300018',
 'PT-S1/2': '#0aa6d8',
 'PT-S3': '#013349',
 'PapE': '#00846f',
 'REN': '#372101',
 'T': '#ffb500',
 'VSMC': '#c2ffed',
 'VSMC/P': '#a079bf',
 'aFIB': '#cc0744',
 'aPT': '#c0b9b2',
 'aTAL1': '#c2ff99',
 'aTAL2': '#001e09',
 'cDC': '#00489c',
 'cycCNT': '#6f0062',
 'cycDCT': '#0cbd66',
 'cycEC': '#eec3ff',
 'cycMNP': '#456d75',
 'cycMYOF': '#b77b68',
 'cycNKC/T': '#7a87a1',
 'cycPT': '#788d66',
 'dC-IC-A': '#885578',
 'dC-TAL': '#fad09f',
 'dCNT': '#ff8a9a',
 'dDCT': '#d157a0',
 'dEC': '#bec459',
 'dEC-PTC': '#456648',
 'dFIB': '#0086ed',
 'dIMCD': '#886f4c',
 'dM-FIB': '#34362d',
 'dM-TAL': '#b4a8bd',
 'dOMCD-PC': '#00a6aa',
 'dPT': '#452c2c',
 'dVSMC': '#636375',
 'ncMON': '#a3c8c9',
 'pDC': '#ff913f',
 'tPC-IC': '#938a81','Unclassified':'#d3d3d3'}

In [4]:
import matplotlib.pyplot as plt
import pickle
import matplotlib.patches as mpatches
from matplotlib.transforms import Bbox

In [ ]:
def df_aver(adata_obs, ann, ann_path, name = 'binary_LISI',csv_folder='20knn_CLISI', mods=['Seurat_WNN', 'snRNA_Multi'], value_metric='median', integrated=False, threshold=0, min_pop_size=15):
    adata_obs[ann] = adata_obs[ann].tolist()
    adata_obs[ann] = adata_obs[ann].astype('category')
    ct_list = adata_obs[ann].cat.categories
    
    df_averages = pd.DataFrame(index=ct_list, columns=[f'{ann}_{mod}' for mod in mods])
    df_counts = pd.DataFrame(index=ct_list, columns=[f'{ann}_{mod}' for mod in mods])
    
    for mod in mods:
        for ct in ct_list:
            ct_sanitized = ct.replace('/', '_')
            filepath = os.path.join(csv_folder, mod, ann_path)
            temp_df = pd.read_csv(filepath + '.csv', index_col=0)
            # print(filepath)
            idxs = adata_obs[ann].isin([ct])
            # print(mod , ct, adata_obs.shape, np.sum(adata_obs.index.isin(idxs)))
            count = np.sum(idxs)
            df_counts.loc[ct, f'{ann}_{mod}'] = count
            
            if count > min_pop_size:
                if value_metric == 'median':
                    average_score = temp_df.loc[idxs][name].median()
                elif value_metric == 'mean':
                    average_score = temp_df.loc[idxs][name].mean()


                if pd.isna(average_score):
                    df_averages.loc[ct, f'{ann}_{mod}'] = 0 
                else:
                    df_averages.loc[ct, f'{ann}_{mod}'] = average_score 
            else:
                # print(f'{ct} in {ann}_{mod} is too small, {count} cells, less than {min_pop_size}')
                df_averages.loc[ct, f'{ann}_{mod}'] = 0
                
    return df_averages, df_counts

In [6]:
adata_obs = pd.read_csv(f'obs/RNA_Integration.csv', index_col=0)

epi_cells = ['FIB', 'VSM/P']
L1_cells = ['FIB', 'VSM/P']
L3_cells = []
for i in L1_cells:
    L3_cells += list(adata_obs[adata_obs['Deepscore_HCA_l1_Clean'].isin([i])]['Deepscore_HCA_l3_Clean'].unique())
L3_cells

['MYOF', 'aFIB', 'FIB', 'VSMC', 'VSMC/P', 'MC', 'REN']

/tmp/ipykernel_11933/492632002.py:39: DtypeWarning: Columns (10,20,21,23,24,25,29,33,35,42,43,54,55,56,57,58,59,62,66,67,69,71) have mixed types. Specify dtype option on import or set low_memory=False.
  adata_obs = pd.read_csv(f'obs/{obs_path}.csv', index_col=0)
/tmp/ipykernel_11933/492632002.py:126: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [ ]:
import matplotlib.lines as mlines

# for mod in mods:
rcParams['figure.figsize'] = (8,8)
cells = L3_cells
colors = l3_colors
shapes = ['o','s', '^', '*', 'D','d','P','1','2','3','4','8','p','+','x']  # Replace with your labels
# Setup
csv_folder = 'knn20_Silhouette'
mods = [ 'Spectral_Atac_mnn','snRNA', 'scRNA', 'scRNA5p', 'RNA_STWG_final', 'Spectral_Multi_mnn', 'MultiVI', 'GLUE']
obs_paths = ['atac','snRNA','scRNA',  'scRNA5p', 'RNA_Integration',  'Vertical', 'MultiVI', 'GLUE_comb']
names = [ 'scMultiome-ATAC Horizontal','scMultiome-RNA Horizontal',"scRNA 3' Horizontal",  "scRNA 5' Horizontal",  'Global RNA Horizontal',  'scMultiome Vertical', 'Global Multimodal Mosaic', 'Global Multimodal Diagonal']

color_dict = {
    'Global RNA Horizontal': '#d62728',
    "scRNA 3' Horizontal": '#1f77b4',
    "scRNA 5' Horizontal": '#ff7f0e',
    'scMultiome-RNA Horizontal': '#2ca02c',
    'scMultiome-ATAC Horizontal': '#8c564b', 
    'scMultiome Vertical': '#9467bd',
    'Global Multimodal Mosaic': '#e377c2',
    'Global Multimodal Diagonal': '#17becf'
}

ann = 'Deepscore_HCA_l1_Clean'
# ann_l = ['Deepscore_HCA_l1_Clean']  # Assuming ann_l is a list of annotations
rcParams['figure.figsize'] = (8, 8)

# Initialize storage for results
results = {}
ann_l = ['Deepscore_HCA_l1_Clean', 'Deepscore_HCA_l3_Clean']
ann = ann_l[1]
df_averages = pd.DataFrame(index=cells, columns=names)
df_counts = pd.DataFrame(index=cells, columns=names)

# Loop over mods, obs_paths, and names
for mod, obs_path, name in zip(mods, obs_paths, names):
    # Read data
    adata_obs = pd.read_csv(f'obs/{obs_path}.csv', index_col=0)
    results[name] = {}

    # Loop over annotations and metrics
    for ann_path in ann_l[0:1]:
        for metric in ['mean']:
            # Compute averages lists
            # averages_list1 = df_aver(adata_obs, ann, ann_path, mods=[mod], value_metric=metric)
            averages_list, counts_list = df_aver(adata_obs, ann, ann_path,name = 'Silhouette', csv_folder=csv_folder, mods=[mod], value_metric=metric, min_pop_size = 15)
            # print(averages_list)
            # Extract values for each cell type
            # epi_cells = ['IC', 'TAL', 'DCT', 'CNT', 'PC', 'PT', 'PEC', 'POD']
            for ct in cells:
                for col in averages_list.columns:
                    counts = counts_list.loc[ct, col]
                    df_counts.at[ct, name] = counts

                    if averages_list.loc[ct, col] == 0:
                        continue
                    val = averages_list.loc[ct, col]
                    # val = (val+1)/2
                    df_averages.at[ct, name] = val
                    results[name][ct] = val
df_averages = df_averages.astype(float)

plt.rcdefaults()
import matplotlib.patches as mpatches


# Calculating the proportions of each cell type within each modality
df_proportions = df_counts.div(df_counts.sum(axis=0), axis=1)

# Averaging the proportions across modalities for each cell type
df_mean_proportions = df_proportions.mean(axis=1)*100


cell_type_colors = [colors[cell] for cell in df_mean_proportions.index]
modality_colors = [color_dict[mod] for mod in df_counts.columns]

import seaborn as sns
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(14, 10))
gs = GridSpec(2, 2, width_ratios=[4, 1], height_ratios=[1, 4], wspace=0.1, hspace=0.1)

# Heatmap
ax_heatmap = fig.add_subplot(gs[1, 0], anchor=(0,0))

sns.heatmap(df_averages.T, cmap='coolwarm', annot=True, ax=ax_heatmap, cbar=False, vmin = -1, vmax=1, fmt = '.2f', annot_kws={'size':14})

# Top bar plot (number of cells per mod)
ax_top_bar = fig.add_subplot(gs[0, 0], anchor = (1,0))
df_mean_proportions.plot(kind='bar', color=cell_type_colors, ax=ax_top_bar)
ax_top_bar.set_ylabel('L1 Cell type %')
ax_top_bar.set_xlabel('')
ax_top_bar.spines['top'].set_visible(False)
ax_top_bar.spines['right'].set_visible(False)
# ax_top_bar.spines['left'].set_visible(False)
ax_top_bar.spines['bottom'].set_visible(False)
ax_top_bar.tick_params(bottom=False, labelbottom=False)  # Remove x-axis labels





# Right bar plot (number of cells per category)
ax_right_bar = fig.add_subplot(gs[1, 1],anchor = (0,1))
df_counts.sum(axis=0)[::-1].plot(kind='barh', stacked=True, color=modality_colors[::-1], ax=ax_right_bar, hatch='//')
ax_right_bar.set_xlabel('Number of Cells')
ax_right_bar.set_ylabel('')
ax_right_bar.spines['top'].set_visible(False)
ax_right_bar.spines['right'].set_visible(False)
ax_right_bar.spines['left'].set_visible(False)
# ax_right_bar.spines['bottom'].set_visible(False)
# Remove unnecessary ticks
ax_top_bar.tick_params(left=False, bottom=False)
ax_right_bar.tick_params(left=False, bottom=False)
ax_right_bar.tick_params(left=False, labelleft=False)  # Remove y-axis labels

cell_type_handles = [mpatches.Patch(color=colors[name], label=name) for name in df_mean_proportions.index]
modality_handles = [mpatches.Patch(color=color_dict[mod], label=mod) for mod in df_counts.columns]

ax_heatmap.legend(handles=cell_type_handles, bbox_to_anchor=(1.31, 1.4), loc='upper left', title="Cell Types")
ax_right_bar.legend(handles=modality_handles, bbox_to_anchor=(1, 0.7), loc='lower left', title="Modalities")

# Adjust layout
ax_top_bar.set_title('Latent Space Silhouette on L1 Annotation over Scenarios', pad=3)
plt.tight_layout()
plt.savefig(f'figures/{csv_folder}_L3_STROMAL.png', dpi = 300, bbox_inches='tight')

atac Deepscore_HCA_l3_Clean Deepscore_HCA_l3_Clean knn20_CLISI mean Spectral_Atac_mnn
         Deepscore_HCA_l3_Clean_Spectral_Atac_mnn
B                                        0.309091
C-TAL                                    0.876662
CCD-IC-A                                 0.888286
CNT                                      0.636138
CNT-IC-A                                 0.121429
CNT-PC                                   0.476886
DCT1                                     0.860072
DCT2                                     0.254858
EC-AEA                                   0.689328
EC-GC                                    0.734747
EC-LYM                                      0.086
EC-PTC                                   0.756058
FIB                                      0.504934
IC-B                                     0.909709
MAC-M2                                   0.331126
MAST                                            0
MC                                       0.582645
MD            

/tmp/ipykernel_24488/819453637.py:39: DtypeWarning: Columns (10,20,21,23,24,25,29,33,35,42,43,54,55,56,57,58,59,62,66,67,69,71) have mixed types. Specify dtype option on import or set low_memory=False.
  adata_obs = pd.read_csv(f'obs/{obs_path}.csv', index_col=0)


MultiVI Deepscore_HCA_l3_Clean Deepscore_HCA_l3_Clean knn20_CLISI mean MultiVI
         Deepscore_HCA_l3_Clean_MultiVI
B                              0.845833
C-TAL                          0.905417
CCD-IC-A                       0.944542
CNT                            0.658743
CNT-IC-A                       0.164815
CNT-PC                         0.542894
DCT1                           0.873677
DCT2                           0.280279
EC-AEA                         0.799227
EC-GC                          0.856551
EC-LYM                         0.764286
EC-PTC                         0.876981
FIB                            0.684454
IC-B                           0.922666
MAC-M2                         0.452083
MAST                           0.518571
MC                             0.612609
MD                             0.613971
MDC                            0.754761
MYOF                               0.17
N                              0.480189
NEU                            0.141667
N

/tmp/ipykernel_24488/819453637.py:128: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [ ]:
import matplotlib.lines as mlines

# for mod in mods:
rcParams['figure.figsize'] = (8,8)
cells = L3_cells
colors = l3_colors
shapes = ['o','s', '^', '*', 'D','d','P','1','2','3','4','8','p','+','x']  # Replace with your labels
# Setup
csv_folder = 'knn20_CLISI'
mods = [ 'Spectral_Atac_mnn','snRNA', 'scRNA', 'scRNA5p', 'RNA_STWG_final', 'Spectral_Multi_mnn', 'MultiVI', 'GLUE']
obs_paths = ['atac','snRNA','scRNA',  'scRNA5p', 'RNA_Integration',  'Vertical', 'MultiVI', 'GLUE_comb']
names = [ 'scMultiome-ATAC Horizontal','scMultiome-RNA Horizontal',"scRNA 3' Horizontal",  "scRNA 5' Horizontal",  'Global RNA Horizontal',  'scMultiome Vertical', 'Global Multimodal Mosaic', 'Global Multimodal Diagonal']

color_dict = {
    'Global RNA Horizontal': '#d62728',
    "scRNA 3' Horizontal": '#1f77b4',
    "scRNA 5' Horizontal": '#ff7f0e',
    'scMultiome-RNA Horizontal': '#2ca02c',
    'scMultiome-ATAC Horizontal': '#8c564b', 
    'scMultiome Vertical': '#9467bd',
    'Global Multimodal Mosaic': '#e377c2',
    'Global Multimodal Diagonal': '#17becf'
}

ann = 'Deepscore_HCA_l1_Clean'
# ann_l = ['Deepscore_HCA_l1_Clean']  # Assuming ann_l is a list of annotations
rcParams['figure.figsize'] = (8, 8)

# Initialize storage for results
results = {}
ann_l = ['Deepscore_HCA_l1_Clean', 'Deepscore_HCA_l3_Clean']
ann = ann_l[1]
df_averages = pd.DataFrame(index=cells, columns=names)
df_counts = pd.DataFrame(index=cells, columns=names)

# Loop over mods, obs_paths, and names
for mod, obs_path, name in zip(mods, obs_paths, names):
    # Read data
    adata_obs = pd.read_csv(f'obs/{obs_path}.csv', index_col=0)
    results[name] = {}

    # Loop over annotations and metrics
    for ann_path in ann_l[1:2]:
        for metric in ['mean']:
            # Compute averages lists
            # averages_list1 = df_aver(adata_obs, ann, ann_path, mods=[mod], value_metric=metric)
            print(obs_path, ann, ann_path, csv_folder, metric, mod)
            averages_list, counts_list = df_aver(adata_obs, ann, ann_path, csv_folder=csv_folder, mods=[mod], value_metric=metric, min_pop_size=15)
            # print(averages_list)
            # Extract values for each cell type
            # epi_cells = ['IC', 'TAL', 'DCT', 'CNT', 'PC', 'PT', 'PEC', 'POD']
            print(averages_list)
            for ct in cells:
                for col in averages_list.columns:
                    counts = counts_list.loc[ct, col]
                    df_counts.at[ct, name] = counts

                    if averages_list.loc[ct, col] == 0:
                        continue
                    val = averages_list.loc[ct, col]
                    # val = (val+1)/2
                    df_averages.at[ct, name] = val
                    results[name][ct] = val
df_averages = df_averages.astype(float)

plt.rcdefaults()
import matplotlib.patches as mpatches


# Calculating the proportions of each cell type within each modality
df_proportions = df_counts.div(df_counts.sum(axis=0), axis=1)

# Averaging the proportions across modalities for each cell type
df_mean_proportions = df_proportions.mean(axis=1)*100


cell_type_colors = [colors[cell] for cell in df_mean_proportions.index]
modality_colors = [color_dict[mod] for mod in df_counts.columns]

import seaborn as sns
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(14, 10))
gs = GridSpec(2, 2, width_ratios=[4, 1], height_ratios=[1, 4], wspace=0.1, hspace=0.1)

# Heatmap
ax_heatmap = fig.add_subplot(gs[1, 0], anchor=(0,0))

sns.heatmap(df_averages.T, cmap='coolwarm', annot=True, ax=ax_heatmap, cbar=False, vmin = 0, vmax=1, fmt = '.2f', annot_kws={'size':10.3})

# Top bar plot (number of cells per mod)
ax_top_bar = fig.add_subplot(gs[0, 0], anchor = (1,0))
df_mean_proportions.plot(kind='bar', color=cell_type_colors, ax=ax_top_bar)
ax_top_bar.set_ylabel('L1 Cell type %')
ax_top_bar.set_xlabel('')
ax_top_bar.spines['top'].set_visible(False)
ax_top_bar.spines['right'].set_visible(False)
# ax_top_bar.spines['left'].set_visible(False)
ax_top_bar.spines['bottom'].set_visible(False)
ax_top_bar.tick_params(bottom=False, labelbottom=False)  # Remove x-axis labels





# Right bar plot (number of cells per category)
ax_right_bar = fig.add_subplot(gs[1, 1],anchor = (0,1))
df_counts.sum(axis=0)[::-1].plot(kind='barh', stacked=True, color=modality_colors[::-1], ax=ax_right_bar, hatch='//')
ax_right_bar.set_xlabel('Number of Cells')
ax_right_bar.set_ylabel('')
ax_right_bar.spines['top'].set_visible(False)
ax_right_bar.spines['right'].set_visible(False)
ax_right_bar.spines['left'].set_visible(False)
# ax_right_bar.spines['bottom'].set_visible(False)
# Remove unnecessary ticks
ax_top_bar.tick_params(left=False, bottom=False)
ax_right_bar.tick_params(left=False, bottom=False)
ax_right_bar.tick_params(left=False, labelleft=False)  # Remove y-axis labels

cell_type_handles = [mpatches.Patch(color=colors[name], label=name) for name in df_mean_proportions.index]
modality_handles = [mpatches.Patch(color=color_dict[mod], label=mod) for mod in df_counts.columns]

ax_heatmap.legend(handles=cell_type_handles, bbox_to_anchor=(1.31, 1.4), loc='upper left', title="Cell Types")
ax_right_bar.legend(handles=modality_handles, bbox_to_anchor=(1, 0.7), loc='lower left', title="Modalities")

# Adjust layout
ax_top_bar.set_title('Latent Space Silhouette on L1 Annotation over Scenarios', pad=3)
plt.tight_layout()
plt.savefig(f'figures/{csv_folder}_L3_STROMAL.png', dpi = 300, bbox_inches='tight')